<a href="https://colab.research.google.com/github/trefftzc/partition_COLAB_notebooks/blob/main/partition_thrust.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Thrust

Thrust is a C++ library that NVIDIA has created to simplify the programming of GPUs.

The web site for Thrust is:
 https://developer.nvidia.com/thrust

 Thrust uses similar design principles to the STL library in C++.

 One of the most used classes in STL is the class vector.

 Thrust extends the class vector with two specialized classes:

 host_vector

 and
  
 device_vector

 host_vector instances are allocated in the memory of the host while device_vector instances are allocated in the memory of the GPU.

 Copying back and forth can be done with a simple assignment.


 The kernels that one uses in CUDA are replaced with functors where one adds the keywords __host__ and/or __device__ to indicate where those functors can be executed.
 functors are C++ functions that can be applied to all the entries in a vector.

 One uses the transform primitive in C++ to apply a functor to all the elements of a vector and then to store the results in another vector.

 Thurst also offers a number of powerful primitives, for instance reduction operations.

In [27]:
!git clone https://github.com/NVIDIA/cccl.git


fatal: destination path 'cccl' already exists and is not an empty directory.


In [28]:
%%writefile partition_thrust.cu
//
// The Thrust documentation is available at
// https://docs.nvidia.com/cuda/thrust/index.html
//
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/sequence.h>
#include <thrust/reduce.h>
#include <thrust/functional.h>
#include <thrust/execution_policy.h>

#include <iostream>



using namespace std;

// In Thrust, kernels are written as functors in C++
// With the additional use of the keywords
// __host__
// __device__
// The keyword __host__ states that this functor can be used on the host
// The keyword __device__ states that this functor can be used on the device


// This functor evaluates a subset encoded by an integer value
// as a possible solution to the partition problem.
// If this subset is indeed a solution, then the result is the
// value of the integer that encodes the subset.
// Otherwise the result is 0.

// Thus the vector with the results will contain non-zero values
// in the entries that encode subsets that are solutions to this
// particular instance of the problem.
// If the instance of the problem does not have any solutions,
// the entire vector with results will contain 0s.

struct evaluateFunctor
{

  const int n;
  const int *array;

  evaluateFunctor(int _n,int *_array) : n(_n),array(_array) {}
    __host__ __device__

    int operator()( const int value) {
      int sum0s = 0;
      int sum1s = 0;
      unsigned int mask = 1;
      for(int i = 0;i < n;i++) {
        if ((mask & value) != 0) {
          sum1s = sum1s + array[i];
        }
        else {
          sum0s = sum0s + array[i];
        }
        mask = mask * 2;
      }
      if (sum0s == sum1s)
        return value;
      else
        return 0;
  }

};

void printResults(int value,int n,thrust::host_vector < int > array)
{
  cout << "Solution:\n" << endl;
  cout << "First partition: " << endl ;
  unsigned int mask = 1;
  int sum = 0;
  for(int i = 0;i < n;i++) {
    if ((mask & value) != 0) {
      cout << array[i] << " ";
      sum = sum + array[i];
    }
    mask = mask * 2;
  }
  cout << " sum: " << sum << endl;
  cout << "Second partition: " << endl ;
  mask = 1;
  sum = 0;
  for(int i = 0;i < n;i++) {
    if ((mask & value) == 0) {
      cout << array[i] << " ";
      sum = sum + array[i];
    }
    mask = mask * 2;
  }
  cout << " sum: " << sum << endl;
}

int main(void) {
    // Read the instance of the problem from standard input
    // Read first the size of the problem

    int n;
    cin >> n;

    // Allocate an array in the host and read the n integer values

    thrust::host_vector < int > host_array(n);
    for(int i = 0;i < n;i++) {
      cin >> host_array[i];
    }

    // Now allocate a device_vector and copy
    // the host_vector to the GPU memory

    thrust::device_vector < int > device_array(n);
    device_array = host_array;

    // Generate the sequence of integer values
    // that encode all the possible subsets

    int powerOf2 = 1;
    for(int i = 0;i < n-1;i++) {
      powerOf2 = powerOf2 * 2;
    }
    std::cout << "Number of subsets to evaluate is : " << powerOf2 << std::endl;
    // is_solution will contain the results of the evaluation
    // It is allocated in the GPU memory
    thrust::device_vector<int>  is_solution(powerOf2);

    // encodingOfSubset will contain the sequence of integer values
    // that encode all the possible subsets
    // It is allocated in the GPU memory
    thrust::device_vector<int> encodingOfSubset(powerOf2);

    //  The sequence function in Thrust initializes the array
    // encodingOfSubset with the sequence of integer values
    // from 0 to powerOf2-1
    thrust::sequence(encodingOfSubset.begin(),encodingOfSubset.end());



    // Instantiate the functor

    evaluateFunctor ef(n,thrust::raw_pointer_cast(device_array.data()));
    // cout << "Instantiated functor " << endl;
    // Execute the kernel. In this case a C++ functor
    // transform applies a functor to the input parameters
    // and leaves the result in the last parameter
    thrust::transform(encodingOfSubset.begin(), encodingOfSubset.end(),
                        is_solution.begin(), ef);
    // cout << "Performed transformation." << endl;
    int result = thrust::reduce(
                            is_solution.begin(),is_solution.end(),
                            0,
                            thrust::maximum<int>());
    // cout << "result is: " << result << endl;
    if (result == 0)
       cout << "This instance does not have a solution. " << endl;
    else
	    printResults(result,n,host_array);

}


Overwriting partition_thrust.cu


In [29]:
!nvcc -Icccl/thrust -Icccl/libcudacxx/include -Icccl/cub partition_thrust.cu -o partition_thrust -arch sm_75


In [30]:
%%writefile test_with_solution_24.txt
24
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 23

Overwriting test_with_solution_24.txt


In [31]:
!time ./partition_thrust < test_with_solution_24.txt

Number of subsets to evaluate is : 8388608
Solution:

First partition: 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1  sum: 23
Second partition: 
23  sum: 23

real	0m0.230s
user	0m0.017s
sys	0m0.210s


In [32]:
%%writefile test_with_no_solution_24.txt
24
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 32

Overwriting test_with_no_solution_24.txt


In [33]:
%%writefile test_with_no_solution_29.txt
29
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 32

Overwriting test_with_no_solution_29.txt


In [34]:
!time ./partition_thrust < test_with_no_solution_29.txt

Number of subsets to evaluate is : 268435456
This instance does not have a solution. 

real	0m0.318s
user	0m0.060s
sys	0m0.255s


In [35]:
!nvprof ./partition_thrust < test_with_no_solution_24.txt


==4490== NVPROF is profiling process 4490, command: ./partition_thrust
Number of subsets to evaluate is : 8388608
This instance does not have a solution. 
==4490== Profiling application: ./partition_thrust
==4490== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   52.50%  647.96us         1  647.96us  647.96us  647.96us  _ZN3cub17CUB_300100_SM_7506detail9transform16transform_kernelINS2_10policy_hubILb1EN4cuda3std3__45tupleIJN6thrust23THRUST_300100_SM_750_NS6detail15normal_iteratorINSA_10device_ptrIiEEEEEEEE10policy1000Ei15evaluateFunctorSF_JPiEEEvT0_iT1_T2_DpNS2_10kernel_argIT3_EE
                   23.76%  293.18us         3  97.727us  2.6560us  147.26us  void cub::CUB_300100_SM_750::detail::for_each::static_kernel<cub::CUB_300100_SM_750::detail::for_each::policy_hub_t::policy_500_t, unsigned long, thrust::THRUST_300100_SM_750_NS::cuda_cub::__uninitialized_fill::functor<thrust::THRUST_300100_SM_750_NS::device_ptr<int

In [36]:
!nvprof ./partition_thrust < test_with_no_solution_29.txt

==4502== NVPROF is profiling process 4502, command: ./partition_thrust
Number of subsets to evaluate is : 268435456
This instance does not have a solution. 
==4502== Profiling application: ./partition_thrust
==4502== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   56.46%  24.083ms         1  24.083ms  24.083ms  24.083ms  _ZN3cub17CUB_300100_SM_7506detail9transform16transform_kernelINS2_10policy_hubILb1EN4cuda3std3__45tupleIJN6thrust23THRUST_300100_SM_750_NS6detail15normal_iteratorINSA_10device_ptrIiEEEEEEEE10policy1000Ei15evaluateFunctorSF_JPiEEEvT0_iT1_T2_DpNS2_10kernel_argIT3_EE
                   21.73%  9.2689ms         3  3.0896ms  2.7200us  4.6358ms  void cub::CUB_300100_SM_750::detail::for_each::static_kernel<cub::CUB_300100_SM_750::detail::for_each::policy_hub_t::policy_500_t, unsigned long, thrust::THRUST_300100_SM_750_NS::cuda_cub::__uninitialized_fill::functor<thrust::THRUST_300100_SM_750_NS::device_ptr<i